In [ ]:
# モデルのダウンロード（githubからclone）
!git clone https://github.com/AliaksandrSiarohin/first-order-model

In [ ]:
# 作業ディレクトリの移動
%cd first-order-model

In [ ]:
# データ格納先ディレクトリの作成
!mkdir uploads/

In [ ]:
# 合成用の画像・動画のダウンロード
from google_drive_downloader import GoogleDriveDownloader as gdd

# 動画（mp4）
gdd.download_file_from_google_drive(file_id='1PaOVqwPdnZkc5YzyqdL_ylCluave9xuU',
                                    dest_path='./uploads/04.mp4',
                                    unzip=False)
# 画像（png）
gdd.download_file_from_google_drive(file_id='1GcG1g8IeJ0ooYqCO-s69S28fX68w5Rn7',
                                    dest_path='./uploads/02.png',
                                    unzip=False)

In [ ]:
# パッケージのインポート
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# display関数の定義
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=33, repeat_delay=1000)
    plt.close()
    return ani

In [ ]:
# 動画をリサイズして生成
from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('./upload/02.png')
driving_video = imageio.mimread('./upload/04.mp4')

# 動画のリサイズ
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='./sample/vox-cpk.pth.tar')

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())